In [1]:
import socket
import time
import pylsl
import csv

# SELECT DATA TO STREAM
acc = True      # 3-axis acceleration
bvp = True      # Blood Volume Pulse
gsr = True      # Galvanic Skin Response (Electrodermal Activity)
tmp = True      # Temperature

serverAddress = '127.0.0.1'
serverPort = 8080
bufferSize = 4096

deviceID = '4e3864' # 'A02088'

# Create CSV file names
acc_data_file = 'acc_data.csv'
bvp_data_file = 'bvp_data.csv'
gsr_data_file = 'gsr_data.csv'
temp_data_file = 'temp_data.csv'

# Create CSV file handlers
acc_data_csv = open(acc_data_file, 'w', newline='')
bvp_data_csv = open(bvp_data_file, 'w', newline='')
gsr_data_csv = open(gsr_data_file, 'w', newline='')
temp_data_csv = open(temp_data_file, 'w', newline='')

acc_data_writer = csv.writer(acc_data_csv)
bvp_data_writer = csv.writer(bvp_data_csv)
gsr_data_writer = csv.writer(gsr_data_csv)
temp_data_writer = csv.writer(temp_data_csv)

def connect():
    global s
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(3)

    print("Connecting to server")
    s.connect((serverAddress, serverPort))
    print("Connected to server\n")

    print("Devices available:")
    s.send("device_list\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))

    print("Connecting to device")
    s.send(("device_connect " + deviceID + "\r\n").encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))

    print("Pausing data receiving")
    s.send("pause ON\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))

connect()
time.sleep(1)

def suscribe_to_data():
    if acc:
        print("Subscribing to ACC")
        s.send(("device_subscribe " + 'acc' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if bvp:
        print("Subscribing to BVP")
        s.send(("device_subscribe " + 'bvp' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if gsr:
        print("Subscribing to GSR")
        s.send(("device_subscribe " + 'gsr' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if tmp:
        print("Subscribing to Temp")
        s.send(("device_subscribe " + 'tmp' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))

    print("Resuming data receiving")
    s.send("pause OFF\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))

suscribe_to_data()
time.sleep(1)

def prepare_LSL_streaming():
    print("Starting LSL streaming")
    if acc:
        infoACC = pylsl.StreamInfo('acc','ACC',3,32,'int32','ACC-empatica_e4');
        global outletACC
        outletACC = pylsl.StreamOutlet(infoACC)
    if bvp:
        infoBVP = pylsl.StreamInfo('bvp','BVP',1,64,'float32','BVP-empatica_e4');
        global outletBVP
        outletBVP = pylsl.StreamOutlet(infoBVP)
    if gsr:
        infoGSR = pylsl.StreamInfo('gsr','GSR',1,4,'float32','GSR-empatica_e4');
        global outletGSR
        outletGSR = pylsl.StreamOutlet(infoGSR)
    if tmp:
        infoTemp = pylsl.StreamInfo('tmp','Temp',1,4,'float32','Temp-empatica_e4');
        global outletTemp
        outletTemp = pylsl.StreamOutlet(infoTemp)

prepare_LSL_streaming()
time.sleep(1)

def reconnect():
    print("Reconnecting...")
    connect()
    suscribe_to_data()
    stream()

def stream():
    try:
        print("Streaming...")
        while True:
            try:
                response = s.recv(bufferSize).decode("utf-8")
                if "connection lost to device" in response:
                    print(response)
                    reconnect()
                    break
                samples = response.split("\n")
                for i in range(len(samples)-1):
                    stream_type = samples[i].split()[0]
                    if stream_type == "E4_Acc":
                        timestamp = float(samples[i].split()[1].replace(',', '.'))
                        data = [int(samples[i].split()[2].replace(',', '.')),
                                int(samples[i].split()[3].replace(',', '.')),
                                int(samples[i].split()[4].replace(',', '.'))]
                        outletACC.push_sample(data, timestamp=timestamp)
                        acc_data_writer.writerow([timestamp] + data)
                    if stream_type == "E4_Bvp":
                        timestamp = float(samples[i].split()[1].replace(',', '.'))
                        data = float(samples[i].split()[2].replace(',', '.'))
                        outletBVP.push_sample([data], timestamp=timestamp)
                        bvp_data_writer.writerow([timestamp, data])
                    if stream_type == "E4_Gsr":
                        timestamp = float(samples[i].split()[1].replace(',', '.'))
                        data = float(samples[i].split()[2].replace(',', '.'))
                        outletGSR.push_sample([data], timestamp=timestamp)
                        gsr_data_writer.writerow([timestamp, data])
                    if stream_type == "E4_Temperature":
                        timestamp = float(samples[i].split()[1].replace(',', '.'))
                        data = float(samples[i].split()[2].replace(',', '.'))
                        outletTemp.push_sample([data], timestamp=timestamp)
                        temp_data_writer.writerow([timestamp, data])
            except socket.timeout:
                print("Socket timeout")
                reconnect()
                break
    except KeyboardInterrupt:
        print("Disconnecting from device")
        s.send("device_disconnect\r\n".encode())
        s.close()
        acc_data_csv.close()
        bvp_data_csv.close()
        gsr_data_csv.close()
        temp_data_csv.close()

stream()


Connecting to server
Connected to server

Devices available:
R device_list 1 | 4E3864 Empatica_E4 

Connecting to device
R device_connect OK

Pausing data receiving
R pause ON

Subscribing to ACC
R device_subscribe acc OK

Subscribing to BVP
R device_subscribe bvp OK

Subscribing to GSR
R device_subscribe gsr OK

Subscribing to Temp
R device_subscribe tmp OK

Resuming data receiving
R pause OFF

Starting LSL streaming
Streaming...
Disconnecting from device
